In [1]:
import glob
from os import path
from PIL import Image
import numpy as np

In [2]:
IMAGE_PATH = 'images_prepped_test' 
file_paths = glob.glob(path.join(IMAGE_PATH, '*.png'))

# Load the images
images_prepped_test = [Image.open(path) for path in file_paths]
images_prepped_test = [img.resize((480, 320),Image.ANTIALIAS) for img in images_prepped_test]
images_prepped_test_np = np.array([np.array(img) for img in images_prepped_test])
images_prepped_test_np.shape

(101, 320, 480, 3)

In [3]:
len(images_prepped_test)

101

In [4]:
images_prepped_test[0].show()

In [5]:
IMAGE_PATH = 'annotations_prepped_train' 
file_paths = glob.glob(path.join(IMAGE_PATH, '*.png'))

# Load the images
annotations_prepped_train = [Image.open(path) for path in file_paths]
annotations_prepped_train = [img.resize((480, 320),Image.ANTIALIAS) for img in annotations_prepped_train]
annotations_prepped_train_np = np.array([np.array(img) for img in annotations_prepped_train])
annotations_prepped_train_np = np.expand_dims(annotations_prepped_train_np, axis=3)
annotations_prepped_train_np.shape

(367, 320, 480, 1)

In [6]:
len(annotations_prepped_train)

367

In [7]:
IMAGE_PATH = 'annotations_prepped_test' 
file_paths = glob.glob(path.join(IMAGE_PATH, '*.png'))

# Load the images
annotations_prepped_test = [Image.open(path) for path in file_paths]
annotations_prepped_test = [img.resize((480, 320),Image.ANTIALIAS) for img in annotations_prepped_test]
annotations_prepped_test_np = np.array([np.array(img) for img in annotations_prepped_test])
annotations_prepped_test_np = np.expand_dims(annotations_prepped_test_np, axis=3)
annotations_prepped_test_np.shape

(101, 320, 480, 1)

In [8]:
len(annotations_prepped_test)

101

In [9]:
annotations_prepped_test[0].show()

In [10]:
IMAGE_PATH = 'images_prepped_train' 
file_paths = glob.glob(path.join(IMAGE_PATH, '*.png'))

# Load the images
images_prepped_train = [Image.open(path) for path in file_paths]
images_prepped_train = [img.resize((480, 320),Image.ANTIALIAS) for img in images_prepped_train]
images_prepped_train_np = np.array([np.array(img) for img in images_prepped_train])
images_prepped_train_np.shape

(367, 320, 480, 3)

In [11]:
len(images_prepped_train)

367

In [3]:
import keras

Using TensorFlow backend.


In [4]:
from keras.preprocessing import image as kImage

In [5]:
from keras.models import Model, Sequential
from keras.layers import Input, UpSampling2D, concatenate, Convolution2D
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
from keras import regularizers

In [6]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(320,480,3))

In [7]:
model = Sequential()
for layer in vgg.layers[:-4]:
  model.add(layer)

In [8]:
for layer in vgg.layers :
  layer.trainable = False

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 320, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 120, 128)      0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 80, 120, 256)     

In [10]:
model.add(UpSampling2D())
model.add(Convolution2D(256, 3, padding='same'))
model.add(Convolution2D(256, 3, padding='same'))
model.add(UpSampling2D())
model.add(Convolution2D(128, 3, padding='same'))
model.add(Convolution2D(128, 3, padding='same'))
model.add(UpSampling2D())
model.add(Convolution2D(64, 3, padding='same'))
model.add(Convolution2D(64, 3, padding='same'))
model.add(UpSampling2D())
model.add(Convolution2D(1, 3, padding='same'))
model.add(Convolution2D(1, 3, padding='same'))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 320, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 120, 128)      0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 80, 120, 256)     

In [11]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [23]:
model.fit(images_prepped_train_np, annotations_prepped_train_np, batch_size=4, epochs=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/2
367/367 [==============================] - 14230s 39s/step - loss: -30.0644 - accuracy: 9.6752e-05
Epoch 2/2
367/367 [==============================] - 13668s 37s/step - loss: -30.5682 - accuracy: 6.7410e-07


In [4]:
from keras.models import load_model

In [25]:
model.save('my_model.h5')

In [13]:
model = load_model('my_model.h5')

ValueError: You are trying to load a weight file containing 18 layers into a model with 0 layers

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 320, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 320, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 160, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 160, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 160, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 80, 120, 128)      0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 80, 120, 256)     

In [16]:
#new approach

In [17]:
inputs = Input(shape=(320,480,3))